In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.f_sale_orders
(
  source_client_id STRING,
  channel STRING,
  order_code STRING,
  cart_id STRING,
  status_name STRING,
  site_code STRING,
  article_id STRING,
  product_barcode STRING,
  unit_code STRING,
  unit_price DOUBLE,
  order_quantity DOUBLE,
  delivered_quantity DOUBLE,
  ordered_amount DOUBLE,
  delivered_amount DOUBLE,
  discount_per_amount DOUBLE,
  net_order_amount DOUBLE,
  vat DOUBLE,
  is_pup_order BOOLEAN,
  customer_phone STRING,
  proc_date TIMESTAMP,
  hash_id BIGINT,
  order_date TIMESTAMP,
  complete_date TIMESTAMP,
  created_date TIMESTAMP,
  pos_code STRING,
  ordercode STRING,
  customerphone STRING
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW view_F_SALE_ORDERS AS
  SELECT DISTINCT * EXCEPT(RN) FROM (
    SELECT 
      * except(HASH_ID), 
      ROW_NUMBER() OVER (PARTITION BY CART_ID, PRODUCT_BARCODE ORDER BY PROC_DATE DESC) AS RN,
      FARM_FINGERPRINT(CONCAT(
        ifnull(CAST(CART_ID AS STRING), ""), 
        ifnull(CAST(PRODUCT_BARCODE AS STRING), ""),
        ifnull(CAST(SITE_CODE AS STRING), ""),
        ifnull(CAST(UNIT_CODE AS STRING), "")
      )) AS HASH_ID
    FROM {catalog_name}.udp_wcm_silver_rop.wp_sale_orders
  ) A 
  WHERE A.RN = 1 
    AND A.SITE_CODE IS NOT NULL 
    AND A.CREATED_DATE IS NOT NULL 
    AND A.ORDER_CODE IS NOT NULL 
    AND A.ARTICLE_ID IS NOT NULL;
""")

In [0]:
spark.sql(f"""
MERGE INTO {catalog_name}.udp_wcm_gold_vcm_dwh.f_sale_orders AS main_tbl
USING (
  SELECT 
    source_client_id,
    channel,
    order_code,
    cart_id,
    status_name,
    site_code,
    article_id,
    product_barcode,
    unit_code,
    unit_price,
    order_quantity,
    delivered_quantity,
    ordered_amount,
    delivered_amount,
    discount_per_amount,
    net_order_amount,
    vat,
    is_pup_order,
    customer_phone,
    to_timestamp(proc_date, 'yyyyMMddHHmmss') AS proc_date,
    HASH_ID,
    to_timestamp(order_date, 'yyyyMMddHHmmss') AS order_date,
    to_timestamp(complete_date, 'yyyyMMddHHmmss') AS complete_date,
    to_timestamp(created_date, 'yyyyMMddHHmmss') AS created_date,
    pos_code
  FROM view_F_SALE_ORDERS
) AS temp_tbl
ON main_tbl.HASH_ID = temp_tbl.HASH_ID

WHEN MATCHED THEN
  UPDATE SET
    main_tbl.source_client_id = temp_tbl.source_client_id,
    main_tbl.channel = temp_tbl.channel,
    main_tbl.order_code = temp_tbl.order_code,
    main_tbl.cart_id = temp_tbl.cart_id,
    main_tbl.status_name = temp_tbl.status_name,
    main_tbl.site_code = temp_tbl.site_code,
    main_tbl.article_id = temp_tbl.article_id,
    main_tbl.product_barcode = temp_tbl.product_barcode,
    main_tbl.unit_code = temp_tbl.unit_code,
    main_tbl.unit_price = temp_tbl.unit_price,
    main_tbl.order_quantity = temp_tbl.order_quantity,
    main_tbl.delivered_quantity = temp_tbl.delivered_quantity,
    main_tbl.ordered_amount = temp_tbl.ordered_amount,
    main_tbl.delivered_amount = temp_tbl.delivered_amount,
    main_tbl.discount_per_amount = temp_tbl.discount_per_amount,
    main_tbl.net_order_amount = temp_tbl.net_order_amount,
    main_tbl.vat = temp_tbl.vat,
    main_tbl.is_pup_order = temp_tbl.is_pup_order,
    main_tbl.order_date = temp_tbl.order_date,
    main_tbl.complete_date = temp_tbl.complete_date,
    main_tbl.created_date = temp_tbl.created_date,
    main_tbl.pos_code = temp_tbl.pos_code,
    main_tbl.customer_phone = temp_tbl.customer_phone,
    main_tbl.proc_date = temp_tbl.proc_date

WHEN NOT MATCHED THEN
  INSERT (
    source_client_id,
    channel,
    order_code,
    cart_id,
    status_name,
    site_code,
    article_id,
    product_barcode,
    unit_code,
    unit_price,
    order_quantity,
    delivered_quantity,
    ordered_amount,
    delivered_amount,
    discount_per_amount,
    net_order_amount,
    vat,
    is_pup_order,
    order_date,
    complete_date,
    created_date,
    pos_code,
    customer_phone,
    proc_date,
    HASH_ID
  )
  VALUES (
    temp_tbl.source_client_id,
    temp_tbl.channel,
    temp_tbl.order_code,
    temp_tbl.cart_id,
    temp_tbl.status_name,
    temp_tbl.site_code,
    temp_tbl.article_id,
    temp_tbl.product_barcode,
    temp_tbl.unit_code,
    temp_tbl.unit_price,
    temp_tbl.order_quantity,
    temp_tbl.delivered_quantity,
    temp_tbl.ordered_amount,
    temp_tbl.delivered_amount,
    temp_tbl.discount_per_amount,
    temp_tbl.net_order_amount,
    temp_tbl.vat,
    temp_tbl.is_pup_order,
    temp_tbl.order_date,
    temp_tbl.complete_date,
    temp_tbl.created_date,
    temp_tbl.pos_code,
    temp_tbl.customer_phone,
    temp_tbl.proc_date,
    temp_tbl.HASH_ID
  );
""")